In [2]:
import numpy as np

In [3]:
M = 20  # kg
g = 9.8 # m / s^2
m = 0.5 # kg
l = 0.5 # m

In [4]:
v1 = ((M + m) * g) / (M * l)
v2 = -(m * g) / M

c1 = -1 / (M * l)
c2 = 1 / M

In [5]:
A = np.array([[0, 1, 0, 0], [0, 0, v2, 0], [0, 0, 0, 1], [0, 0, v1, 0]])
B = np.array([[0], [c2], [0], [c1]])

In [6]:
print(np.shape(A))
print(np.shape(B))

(4, 4)
(4, 1)


## Controllability

CM = [B AB A<sup>2</sup>B A<sup>3</sup>B]

In [7]:
CM = np.array([[B], [A @ B], [A @ A @ B], [A @ A @ A @ B]]).reshape(4, 4)

In [8]:
print((CM))

[[ 0.      0.05    0.     -0.1   ]
 [ 0.05    0.     -0.1     0.    ]
 [ 0.      0.0245  0.     -2.009 ]
 [ 0.0245  0.     -2.009   0.    ]]


In [9]:
rank = np.linalg.matrix_rank(CM)
print("Rank of CM: ", rank)

Rank of CM:  4


> Rank of CM = 4, therefore controllable

## Stability


In [10]:
print(A)

[[ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.245  0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.    20.09   0.   ]]


In [11]:
eigenvalues = np.linalg.eigvals(A)

print("Eigenvalues of matrix A:", eigenvalues)

Eigenvalues of matrix A: [ 0.          0.          4.48218697 -4.48218697]


>One eigen value is positive, so the system is unstable

## Policy Iteration


In [12]:
Q = np.diag([0, 100, 100, 100])
R = np.array([[0.1]])


In [13]:
print("A:\n", A)
print("B:\n", B)

A:
 [[ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.245  0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.    20.09   0.   ]]
B:
 [[ 0.  ]
 [ 0.05]
 [ 0.  ]
 [-0.1 ]]


In [14]:
F = [   [0.  ]  ,       [-2.56725429], [-233.98715586],  [-52.22286859]]

F = np.array(F).reshape(1, 4)
print("Eigenvalues of A + BF:", np.linalg.eigvals(A + B @ F))

Eigenvalues of A + BF: [ 0.          9.63475861 -0.05825404 -4.48258043]


In [15]:
import control
# Define the desired closed-loop poles
desired_poles = np.array([-20, 10, -3, -2])

# Compute the feedback gain matrix F that places the closed-loop poles at the desired locations
F = control.place(A, B, desired_poles)

# Check if the closed-loop system is stable
if np.all(np.real(np.linalg.eigvals(A + B @ F)) < 0):
    print("Closed-loop system is stable")
else:
    print("Closed-loop system is unstable")


Closed-loop system is unstable


## Hewer's Algorithm

In [16]:
from scipy.linalg import solve_continuous_are

def policy_iteration(A, B, Q, R, F):
    # P is I in 4x4
    P = solve_continuous_are(A, B, Q, R) # solves the continuous algebraic Riccati equation

    
    for i in range(100):
        F = -np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A
        P = Q + F.T @ R @ F + (A + B @ F).T @ P @ (A + B @ F)

    return F

 
F = policy_iteration(A, B, Q, R, F)

print("F", F)

F [[0.00000000e+000 0.00000000e+000 2.00400816e+002 2.50905771e-126]]


In [17]:
F = [   [0.  ]  ,       [-2.56725429], [-233.98715586],  [-52.22286859]]
F = np.array(F).reshape(1, 4)


In [18]:
import cv2

class InvertedPendulum:
    def __init__(self):
        f = 0

    def step( self, state_vec, t=None ):
        """ state vector :
                x0 : position of the cart
                x1 : veclocity of the cart
                x2 : angle of pendulum. In ref frame with x as forward of the cart and y as up. Angile with respect to ground plane
                x3 : angular velocity of the pendulum
        """
        CART_POS = state_vec[0]
        BOB_ANG  = state_vec[2]*180. / np.pi  # degrees
        LENGTH_OF_PENDULUM = 110.

        IM = np.zeros( (512, 512,3), dtype='uint8' )
        # IM = cv2.imread('imgs/background.jpg').resize((512, 512))

        # Ground line
        cv2.line(IM, (0, 450), (IM.shape[1], 450), (100,20,40), 4 )


        # Mark ground line
        XSTART = -5.
        XEND = 30.
        for xd in np.linspace( XSTART, XEND, 11 ):
            x = int(   (xd - XSTART) / (XEND - XSTART) * IM.shape[0]   )

            cv2.circle( IM, (x, 450), 5, (0,0,255), -1 )

            cv2.putText(IM, str(xd), (x-15,450+15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (100,255,10), 1)


        # Draw Wheels of the cart
        wheel_1_pos = int(   (CART_POS - 2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        wheel_2_pos = int(   (CART_POS + 2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.circle( IM, (wheel_1_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_2_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_1_pos, 415), 2, (255,255,255), -1 )
        cv2.circle( IM, (wheel_2_pos, 415), 2, (255,255,255), -1 )

        # Cart base
        cart_base_start = int(   (CART_POS - 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        cart_base_end   = int(   (CART_POS + 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.line( IM, (cart_base_start, 380), (cart_base_end, 380), (100,100,100), 6 )

        # Pendulum hinge
        pendulum_hinge_x = int(   (CART_POS - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        pendulum_hinge_y = 380
        cv2.circle( IM, (pendulum_hinge_x, pendulum_hinge_y), 10, (100,100,100), -1 )


        # Pendulum
        pendulum_bob_x = int( LENGTH_OF_PENDULUM * np.cos( BOB_ANG / 180. * np.pi ) )
        pendulum_bob_y = int( LENGTH_OF_PENDULUM * np.sin( BOB_ANG / 180. * np.pi ) )
        cv2.circle( IM, (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), 10, (255,255,255), -1 )
        cv2.line( IM, (pendulum_hinge_x, pendulum_hinge_y), (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), (255,255,255), 3 )

        # Mark the current angle
        angle_display = BOB_ANG % 360
        if( angle_display > 180 ):
            angle_display = -360+angle_display
        cv2.putText(IM, "theta="+str( np.round(angle_display,4) )+" deg", (pendulum_hinge_x-15, pendulum_hinge_y-15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)


        # Display on top
        if t is not None:
            cv2.putText(IM, "t="+str(np.round(t,4))+"sec", (15, 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,0), 1)
            cv2.putText(IM, "ANG="+str(np.round(BOB_ANG,4))+" degrees", (15, 35), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,0), 1)
            cv2.putText(IM, "POS="+str(np.round(CART_POS,4))+" m", (15, 55), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,0), 1)

        return IM

In [19]:
sys = InvertedPendulum()

# Initial state
x0 = np.array([[0], [0], [np.pi/6], [0.]])

# U = F @ x, loop and store U and x
x = x0
U = []
X = []

for i in range(1000):
    u = F @ x
    x_dot = A @ x + B @ u
    
    x = x + x_dot * 0.01

    U.append(u)
    X.append(x)

In [20]:
# print eig values of A + BF
print("Eigen values of A + BF", np.linalg.eigvals(A + B @ F))

Eigen values of A + BF [ 0.          9.63475861 -0.05825404 -4.48258043]


In [21]:
syst = InvertedPendulum()

for i in range(1000):
    rendered = syst.step( [X[i][0], X[i][1], X[i][2], X[i][3]], t=i*0.01)
    print(X[i][2])
    cv2.imshow( 'im', rendered )

    # Press q to exit

    if cv2.waitKey(5) == ord('q'):
        break

cv2.destroyAllWindows()

[0.52359878]
[0.52587584]
[0.53054728]
[0.53774627]
[0.54762322]
[0.56034718]
[0.57610747]
[0.59511548]
[0.61760659]
[0.64384235]
[0.67411281]
[0.70873915]
[0.7480765]
[0.79251714]
[0.84249387]
[0.89848385]
[0.96101272]
[1.03065918]
[1.10805999]
[1.19391545]
[1.28899543]
[1.39414599]
[1.5102966]
[1.63846813]
[1.77978155]
[1.9354675]
[2.10687677]
[2.29549185]
[2.50293948]
[2.73100455]
[2.98164523]
[3.25700962]
[3.55945402]
[3.89156288]
[4.25617077]
[4.65638642]
[5.09561909]
[5.57760747]
[6.10645139]
[6.68664656]
[7.32312267]
[8.02128519]
[8.78706117]
[9.62694941]
[10.54807559]
[11.55825257]
[12.66604663]
[13.88084996]
[15.21296026]
[16.67366797]
[18.27535183]
[20.03158376]
[21.9572438]
[24.06864609]
[26.38367704]
[28.9219468]
[31.70495532]
[34.75627439]
[38.10174726]
[41.76970746]
[45.79121873]
[50.20033803]
[55.03440396]
[60.33435287]
[66.14506552]
[72.51574708]
[79.50034378]
[87.15799969]
[95.5535575]
[104.75810753]
[114.84958972]
[125.91345354]
[138.04338149]
[151.34208235]
[165.9221

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type


In [ ]:
cv2.destroyAllWindows()

: 

: 